<a href="https://colab.research.google.com/github/KANG-HYUNIL/AudioTrain/blob/main/AudioTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Explanation

This notebook demonstrates a process for audio classification, including data loading, preprocessing, data augmentation, building and training deep learning models (CNN and CRNN), and evaluating their performance using K-Fold cross-validation.

## Setup and Environment

This section installs necessary packages, sets up the environment, and connects to Google Drive to access data.

In [ ]:
#우선 colab에서 실험 및 실습할 때에 필요한 패키지들을 설치해보자
!pip -q install librosa torchaudio soundfile ipywidgets numpy pandas tqdm tensorflow tensorflow-datasets scikit-learn mirdata
!apt-get update
!apt-get install -y sox libsox-dev libsox-fmt-all

import os
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import IPython.display as ipd
import soundfile as sf
import torchaudio

import os, glob, numpy as np, pandas as pd
import librosa, soundfile as sf
from tqdm import tqdm
import torch
import random
from torchaudio import sox_effects
import tensorflow_datasets as tfds

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import itertools
from pathlib import Path
import tensorflow.keras


# Google Drive 연결 및 Kaggle 용 폴더 만들기


from google.colab import drive
drive.mount('/content/drive')

This cell verifies GPU availability and configures TensorFlow to use memory growth, which helps in managing GPU memory more efficiently during training.

In [ ]:
# GPU 확인
torch.cuda.is_available()
torch.cuda.current_device()
torch.cuda.device_count()
torch.cuda.get_device_name(0)

import tensorflow as tf

# 1) GPU 장치 확인 + 메모리 성장
gpus = tf.config.list_physical_devices('GPU')
print("GPUs:", gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)  # 점진 할당

This cell demonstrates loading and visualizing a sample audio file using the `librosa` library. It shows how to load a `.wav` file, resample it, display its waveform, and play the audio.

In [ ]:
# 실제로 wav 파일 load해보기


# # wav 들어있는 경로들 중 하나
# preprocessed_data = Path("/content/drive/MyDrive/kaggle_cache/tf_speech_cmds/preprocessed_data")

# 전처리해서 npy로 변환한 데이터만 google drive에 있기에, librosa 예제로 해보자, 샘플 파일이 있다고 하네
wav_path = librosa.ex('trumpet')

#librosa.load로 wav를 load하는 과정
#sr은 샘플레이트 Hz 설정. 이전에 샘플레이트 개념 간단하게라도 보앗엇지?
# 1초에 얼마나 많은 데이터를 담아둘 지 정하는 수치. sr를 load메서드에 인자로 넣으면, path에 있는 데이터를 알아서 sr수치로 리샘플링 한다고
# 근데 그러면 데이터의 길이가 줄어드는건가? 흠
y, sr = librosa.load(wav_path, sr=22050, mono=True)
print(f"Librosa Loaded: y.shape={y.shape}, sr={sr}")

# 받아온 파형을 한 번 시각화해보자
plt.figure(figsize = (10, 3))
librosa.display.waveshow(y, sr=sr)
plt.show()


#ipd를 통해 실제 듣기도 가능하다고
ipd.Audio(y, rate=sr)

This section defines functions and parameters for data augmentation techniques applied to audio files. It includes settings for output directories, augmentation probabilities, and parameter ranges for effects like gain, noise, time stretch, pitch shift, EQ, and reverb. Utility functions for loading and normalizing audio are also included.

In [ ]:
# Data Augmentation 진행해보자

# Wav, 즉 음원 파일 자체에 좀 변경을 가하는 것들로만 우선./
import random

# =========================================
# [셀 2] 경로 & 하이퍼파라미터 설정
# =========================================
DATA_RAW_DIR = "/content/drive/MyDrive/audioPractice/raw"      # 입력: class별 폴더에 .wav
DATA_AUG_DIR = "/content/drive/MyDrive/audioPractice/aug"      # 출력: 증강된 wav 저장
META_DIR      = "/content/drive/MyDrive/audioPractice/meta"     # 출력: train/val CSV



# 미리 경로 생성
os.makedirs(DATA_AUG_DIR, exist_ok=True)
os.makedirs(META_DIR, exist_ok=True)


# 증강 개수: 각 원본 파일당 생성할 증강본 수
AUG_PER_FILE = 2

# 학습/검증 분할 비율
VAL_RATIO = 0.2

# 공통 샘플레이트(통일 권장). None이면 원본 SR 유지
TARGET_SR = 22050


# 랜덤 증강 확률 (필요에 맞게 조정)
P_GAIN      = 0.7
P_NOISE     = 0.7
P_STRETCH   = 0.5
P_PITCH     = 0.5
P_EQ        = 0.5
P_REVERB    = 0.5


# 파라미터 범위
GAIN_DB_RANGE       = (-6, 6)            # dB
NOISE_SNR_DB_CHOICES = [30, 20, 10]      # dB
STRETCH_RATE_RANGE  = (0.9, 1.1)         # <1 느리게, >1 빠르게
PITCH_STEPS_CHOICES = [-2, -1, 1, 2]     # 반음


# SoX EQ: ["equalizer", center_freq(Hz), Q, gain(dB)]
EQ_PRESETS = [
    ["equalizer", "120", "1.0q", "-6"],
    ["equalizer", "1000", "1.0q", "+3"],
    ["equalizer", "4000", "1.0q", "+2"],
]
# SoX Reverb: ["reverb", reverberance, HF_damping, room_scale]
REVERB_PRESETS = [
    ["reverb", "30", "30", "50"],
    ["reverb", "50", "50", "70"],
    ["reverb", "70", "50", "90"],
]


# =========================================
# [셀 3] 유틸 함수 (오디오 I/O, 정규화 등)
# =========================================
def load_mono(path, target_sr=TARGET_SR):
    y, sr = librosa.load(path, sr=target_sr, mono=True)
    return y.astype(np.float32), sr

def peak_normalize(y, eps=1e-9):
    peak = np.max(np.abs(y))
    if peak < eps:
        return y
    return (y / (peak + eps)).astype(np.float32)



# =========================================
# [셀 4] 증강 함수 (WAV 레벨)
# =========================================
def aug_gain(y):
    db = np.random.uniform(*GAIN_DB_RANGE)
    g  = 10**(db/20.0)
    return np.clip(y * g, -1.0, 1.0)

def aug_noise(y, snr_db=None):
    if snr_db is None:
        snr_db = random.choice(NOISE_SNR_DB_CHOICES)
    sig_pwr = np.mean(y**2) + 1e-12
    noise_pwr = sig_pwr / (10**(snr_db/10))
    noise = np.random.normal(0.0, np.sqrt(noise_pwr), size=y.shape).astype(np.float32)
    return np.clip(y + noise, -1.0, 1.0)

def aug_time_stretch(y, sr):
    rate = np.random.uniform(*STRETCH_RATE_RANGE)
    # librosa time-stretch는 길이가 변함 (라벨 동기화 필요할 수 있음)
    y2 = librosa.effects.time_stretch(y, rate=rate).astype(np.float32)
    return y2

def aug_pitch_shift(y, sr):
    steps = random.choice(PITCH_STEPS_CHOICES)
    y2 = librosa.effects.pitch_shift(y, sr=sr, n_steps=steps).astype(np.float32)
    return y2

def to_torch_wave(y):
    # torchaudio sox_effects는 [C, T] 텐서 필요
    return torchaudio.functional.resample(
        torch.tensor(y).unsqueeze(0), orig_freq=sr, new_freq=sr
    )


def run_sox_effects_numpy(y, sr, effects_chain):
    """
    torchaudio sox_effects를 numpy(y) 입력에 적용하고 numpy로 반환
    """
    wav_t = torch.from_numpy(y).unsqueeze(0)  # [1, T]
    out, out_sr = sox_effects.apply_effects_tensor(wav_t, sr, effects_chain)
    out = out.squeeze(0).numpy().astype(np.float32)
    return out, out_sr

# 수정 제안: EQ 효과 뒤에 채널 효과 추가
def aug_eq(y, sr):
    # EQ 프리셋 중 랜덤 1~2개 적용
    n = random.choice([1, 2])
    eq_chain = random.sample(EQ_PRESETS, n)

    # 채널을 1로 강제하는 효과 추가
    # EQ 효과 뒤에 추가합니다.
    final_chain = eq_chain + [["channels", "1"]]

    out, _ = run_sox_effects_numpy(y, sr, final_chain)
    return out

def aug_reverb(y, sr):
    chain = [random.choice(REVERB_PRESETS)]
    # 채널을 1로 강제하는 효과 추가
    chain.append(["channels", "1"])
    out, _ = run_sox_effects_numpy(y, sr, chain)
    return out



# =========================================
# [셀 5] 랜덤 증강 파이프라인
# =========================================
def random_augment(y, sr):
    """
    - 확률적으로 여러 증강을 '연속 적용' (순서 중요 X, 간단 버전)
    - 마지막에 peak normalize로 출력 안전하게
    """
    out = y.copy()

    if random.random() < P_GAIN:
        out = aug_gain(out)

    if random.random() < P_NOISE:
        out = aug_noise(out)

    # if random.random() < P_STRETCH:
    #     out = aug_time_stretch(out, sr)

    if random.random() < P_PITCH:
        out = aug_pitch_shift(out, sr)

    if random.random() < P_EQ:
        out = aug_eq(out, sr)

    if random.random() < P_REVERB:
        out = aug_reverb(out, sr)

    out = peak_normalize(out)
    return out


# =========================================
# [셀 6] 증강 실행:
# =========================================
def ensure_dir(path):
    os.makedirs(path, exist_ok=True)



# 한 wav를 받아서 그 wav에 대해 Augmenatation하고 y와 sr 반환
def data_augmentation(y, sr):
  y_aug = random_augment(y, sr)
  return y_aug, sr

This function converts audio waveforms into Mel spectrograms, which are commonly used features for audio classification tasks. It uses `librosa` to perform Short-Time Fourier Transform (STFT) and compute the Mel spectrogram on a decibel scale.

In [ ]:
#Mel Spectrogram 변환 후 Npy변환

def create_mel_spectrogram(y, sr, n_fft=2048, hop_length = 512, n_mels = 128):
    """
    오디오 파형(y)과 샘플링 속도(sr)를 받아 멜 스펙트로그램을 계산합니다.

    Args:
        y (np.ndarray): 오디오 파형 데이터 (float32).
        sr (int): 오디오의 샘플링 속도.
        n_fft (int): FFT 윈도우 크기.
        hop_length (int): 홉 길이 (프레임 간의 샘플 수).
        n_mels (int): 멜 밴드의 수.

    Returns:
        np.ndarray: 계산된 멜 스펙트로그램 (NumPy 배열).
    """

    # STFT, short time fourier transform 계산, 결괏값은 복소수 Spectrum들의 합쳐진 값인 Spectrogram
    D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)

    # 진폭, Amplitude Spectrogram으로 변환
    S_amplitude = np.abs(D)

        # 멜 스펙트로그램으로 변환
    # librosa.feature.melspectrogram은 파워 스펙트로그램(amplitude**2) 또는 진폭 스펙트로그램을 입력받을 수 있습니다.
    # 여기서는 진폭 스펙트로그램을 사용합니다. power=2.0으로 설정하면 파워 스펙트로그램을 사용하게 됩니다.
    mel_spectrogram = librosa.feature.melspectrogram(S=S_amplitude, sr=sr, n_mels=n_mels, hop_length=hop_length, n_fft=n_fft)

      # (옵션) 데시벨(dB) 스케일로 변환 (사람의 청각은 로그 스케일에 가깝기 때문에 흔히 사용됩니다)
    # ref=np.max 는 스펙트로그램의 최대값을 기준으로 정규화합니다.
    mel_spectrogram_db = librosa.amplitude_to_db(mel_spectrogram, ref=np.max)

    # 일반적으로 dB 스케일의 멜 스펙트로그램을 특징으로 많이 사용합니다.
    return mel_spectrogram_db

This cell defines functions for encoding categorical labels (instrument names in this case) into numerical indices and one-hot encoded vectors, which are required for training classification models.

In [ ]:
#One-Hot Encoder 생성 코드

import numpy as np

# IRMAS 고정 클래스(훈련 기준 11개)
INST_CODES = ["cel","cla","flu","gac","gel","org","pia","sax","tru","vio","voi"]
inst2idx = {c:i for i,c in enumerate(INST_CODES)}
idx2inst = {i:c for c,i in inst2idx.items()}

def label_to_index(code: str) -> int:
    return inst2idx[code]  # KeyError가 싫으면 inst2idx.get(code, default)로

def index_to_onehot(idx: int, num_classes=len(INST_CODES)) -> np.ndarray:
    return np.eye(num_classes, dtype=np.float32)[idx]

def code_to_onehot(code: str) -> np.ndarray:
    return index_to_onehot(label_to_index(code))

# 사용 예시
y_idx = label_to_index("gac")        # 3
y_1h  = code_to_onehot("gac")        # [0,0,0,1,0, ...]

This cell installs SoX, a command-line utility for audio processing, which is used by the `torchaudio.sox_effects` module for some of the data augmentation techniques.

In [ ]:
# Install SoX
!apt-get update
!apt-get install -y sox libsox-dev libsox-fmt-all

This cell is intended for downloading and processing a dataset (IRMAS in this case). It iterates through the dataset, loads audio files, applies data augmentation (using the functions defined previously), converts the augmented audio to Mel spectrograms, and saves the features and corresponding one-hot encoded labels as `.npy` files to Google Drive. It includes logic to limit the number of data points processed per class.

In [ ]:
# DataSet 다운로드 하는 함수

import mirdata
from pathlib import Path

# 증강 개수: 각 원본 파일당 생성할 증강본 수
AUG_PER_FILE = 2

META_DIR = "/content/drive/MyDrive/audioPractice/meta"

DATA_HOME = Path("/content/IRMAS")   # 저장 위치(드라이브 마운트 경로도 가능)

# ds = mirdata.initialize("irmas", data_home=str(DATA_HOME))
# # 훈련만 받기: 키 이름은 mirdata에 하드코딩되어 있습니다.
# # ['training_data','testing_data_1','testing_data_2','testing_data_3']
# ds.download(partial_download=["training_data"], force_overwrite=True)

# Class 별 이용할 데이터 수(데이터셋에서)
DATA_PER_CLASS = 100

class_count = {}

#각 세트에서 300개씩만 처리하자
tracks = ds.load_tracks()

#tid = 악기 id, tr = 악기 Track 객체
for tid, tr in tracks.items():
    if not tr.train:    # 안전장치: 훈련용만
        continue
    wav_path = tr.audio_path                 # .wav 절대경로
    label_codes = tr.instrument              # ['gac']처럼 리스트
    label = label_codes[0]                   # 훈련셋은 단일 라벨

    # npy 의 Class 분류 위한 폴더 경로 생성
    output_dir = Path(META_DIR) / label
    os.makedirs(output_dir, exist_ok=True)

    #class count 추가
    class_count[label] = class_count.get(label, 0) + 1

    #이미 Class 별 데이터 수가 DATA_PER_CLASS를 넘었으면 Skip
    if class_count[label] > DATA_PER_CLASS:
        continue

    # 이후 npy 저장을 위한 filename 추출
    original_filename = os.path.basename(tr.audio_path)
    base_filename = os.path.splitext(original_filename)[0] # 확장자 제거

    # Label Encoidng=
    label_oneHot = code_to_onehot(label)

    #Mono로 Load하기
    y, sr = load_mono(path=wav_path)

    #원본 먼저 변환하고 저장해놓기
    mel_spectrogram = create_mel_spectrogram(y=y, sr=sr)

    #이제 저장해야지
    features_path = os.path.join(output_dir, f"{base_filename}_features.npy")
    label_path = os.path.join(output_dir, f"{base_filename}_label.npy")

    # 데이터 저장
    np.save(features_path, mel_spectrogram)
    np.save(label_path, label_oneHot) # 라벨도 .npy로 저장



    # Mel Spectrogram 획득
    for i in range(1, AUG_PER_FILE + 1):

        #Augmentation 진행
        y_aug, sr_aug = data_augmentation(y, sr)

        #mel Spectrogram 획득
        mel_spectrogram = create_mel_spectrogram(y_aug, sr_aug)



        #이제 저장해야지
        features_path = os.path.join(output_dir, f"{base_filename}_{i}_features.npy")
        label_path = os.path.join(output_dir, f"{base_filename}_{i}_label.npy")

        # 데이터 저장
        np.save(features_path, mel_spectrogram)
        np.save(label_path, label_oneHot) # 라벨도 .npy로 저장

This section handles the process of copying the preprocessed `.npy` feature and label files from Google Drive to the local Colab instance's SSD. This is done to speed up data loading during model training. It uses the `rsync` command for efficient copying.

In [ ]:
# Google Drive의 파일을 실제 Colab 인스턴스의 ssd로 다운받아두는 작업
#

# 실제 Google Drive에 npy들 저장되어 있는 경로
META_DIR = "/content/drive/MyDrive/audioPractice/meta"

# Colab 인스턴스 저장 경로
LOCAL_DIR = "/content/data"

# Google Drive 데이터 복사

import os
import subprocess
from tqdm import tqdm

# 로컬 저장소 디렉토리가 없으면 생성
os.makedirs(LOCAL_DIR, exist_ok=True)

# Google Drive에서 로컬로 파일 복사
# rsync 명령어를 사용하여 복사합니다. -a: 아카이브 모드, -v: 상세 출력, --progress: 진행 상황 표시
# --ignore-existing: 이미 로컬에 파일이 존재하면 건너뛰어 시간을 절약합니다.
# Google Drive 경로 끝에 /를 붙이면 해당 디렉토리의 내용물을, 안 붙이면 디렉토리 자체를 복사합니다.
# 여기서는 내용물을 복사하므로 META_DIR 뒤에 /를 붙입니다.
print(f"Copying data from {META_DIR} to {LOCAL_DIR}...")

# tqdm과 연동하여 진행률 표시
# rsync의 --info=progress2 옵션은 전체 복사 크기와 현재까지 복사된 크기를 출력합니다.
# 이를 파싱하여 tqdm으로 진행률을 업데이트합니다.
# 주의: 이 방법은 rsync 출력이 특정 형식일 때만 작동하며, 환경에 따라 다를 수 있습니다.
# 간단하게는 !rsync -av --progress "{META_DIR}/" "{LOCAL_DIR}/" 만 사용해도 됩니다.

# rsync 명령 실행
process = subprocess.Popen(
    ['rsync', '-av', '--progress', f'{META_DIR}/', f'{LOCAL_DIR}/'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True
)

# 진행 상황 파싱 및 tqdm 업데이트
# 이 부분은 rsync의 실제 출력 형식을 보고 조정해야 할 수 있습니다.
# 간단한 복사 진행률 확인을 위해 stdout을 실시간으로 읽습니다.
# 더 정확한 tqdm 연동은 rsync 출력을 정규식 등으로 파싱해야 합니다.

# 간단한 진행 메시지만 출력합니다.
# 정확한 tqdm 진행률은 구현이 복잡하므로, 여기서는 rsync 자체의 --progress 출력을 사용합니다.
!rsync -av --progress "{META_DIR}/" "{LOCAL_DIR}/"

print("Data copy complete.")

This cell prepares the data for training by creating a `tf.data.Dataset` from the local `.npy` files. It defines a function `_load_npy` to load the feature and label arrays and ensures they have the correct shape and data type. It then configures the dataset for shuffling, parallel loading, and padding batches to handle variable-length time series data.

In [ ]:
import tensorflow as tf
import numpy as np


import os
import glob
import numpy as np
from tqdm import tqdm

#DataLoader 및 Generator 만들 예정 부분 셀 코드

# npy 파일이 저장된 루트 경로
# META_DIR = "/content/drive/MyDrive/audioPractice/meta" # Google Drive 경로는 이제 사용하지 않습니다.

# 로컬 저장소 경로를 사용합니다.
LOCAL_DIR = "/content/data"


def collect_pairs(meta_root):
    pairs = []
    for root, _, _ in os.walk(meta_root):
        for f in sorted(glob.glob(os.path.join(root, "*_features.npy"))):
            base = f[:-len("_features.npy")]
            lf = base + "_label.npy"
            if os.path.exists(lf):
                pairs.append((f, lf))
            else:
                print(f"[WARN] label missing for {f}")
    return pairs

# collect_pairs 함수 호출 시 LOCAL_DIR 사용
pairs = collect_pairs(LOCAL_DIR)
print("num pairs:", len(pairs))



#넌 뭐야? 각 경로 쌍 리스트 만들어두기!
feature_paths = [p[0] for p in pairs]
label_paths   = [p[1] for p in pairs]

# 한 번에 모델로 들어갈 Batch 크기
BATCH_SIZE = 32



#DataSet 구성
ds = tf.data.Dataset.from_tensor_slices((feature_paths, label_paths))
ds_initial = tf.data.Dataset.from_tensor_slices((feature_paths, label_paths))

#npy load하는 메서드
def _load_npy(feat_path, lab_b):


    def py_load(feat_b, lab_b):

        #실제 npy 읽기
        fx = np.load(feat_b.decode("utf-8"))        # (F, T[, C]) 형태 가정
        y  = np.load(lab_b.decode("utf-8"))         # (num_classes,) 혹은 (,) class idx

        # 형/dtype 정리
        if fx.ndim == 2:        # 채널 축 없으면 추가
            fx = fx[..., np.newaxis]

        #float 32로
        fx = fx.astype(np.float32, copy=False)

        y  = y.astype(np.float32, copy=False) if y.ndim>0 else np.int64(y)  # 원핫이면 float32, 인덱스면 int

        return fx, y

    x, y = tf.numpy_function(py_load, [feat_path, lab_b],
                             [tf.float32, tf.float32])  # 인덱스 라벨이면 tf.int64로 바꿔도 됨

    # 부분 shape 고정(Conv2D는 rank/일부 축 정보 필요)
    # 시간 축은 가변이므로 None으로 둡니다.
    x.set_shape([128, None, 1])   # (F, T, C) — F=128, C=1 고정, T 가변
    # 라벨 shape 고정 (원핫 인코딩된 11개 클래스)
    y.set_shape([11])
    return x, y

# (선택) 속도 우선이면 비결정성 허용
options = tf.data.Options()
options.deterministic = False  # 재현성이 더 중요하면 이 줄 제거


ds = (ds
      .shuffle(len(pairs)) # 매 epoch마다 random 순서
      .map(_load_npy, num_parallel_calls=tf.data.AUTOTUNE) # 병렬 로딩
      # padded_batch를 사용하여 가변 길이를 처리합니다.
      # padded_shapes: 각 텐서의 패딩될 모양을 지정합니다. None은 해당 축이 가변적임을 나타냅니다.
      # padding_values: 패딩에 사용할 값을 지정합니다. 0.0은 0으로 패딩합니다.
      .padded_batch(BATCH_SIZE,
                    padded_shapes=([128, None, 1], [11]),
                    padding_values=(tf.constant(0.0, dtype=tf.float32), tf.constant(0.0, dtype=tf.float32)),
                    drop_remainder=False) # Batch 묵기
      .with_options(options)
      .prefetch(tf.data.AUTOTUNE))

This cell performs a quick check on the loaded `.npy` files to confirm their shapes and data types, ensuring that the data loading and preprocessing steps were successful before feeding the data into a model.

In [ ]:
import numpy as np
import random
import os

# pairs 리스트는 이미 이전 셀에서 생성되었다고 가정합니다.
# 만약 pairs 리스트가 정의되지 않았다면, 이전 셀(3d184f11)을 먼저 실행해야 합니다.
if 'pairs' not in locals():
    print("Error: 'pairs' list not found. Please run the cell that collects file pairs first.")
else:
    print(f"Total number of pairs found: {len(pairs)}")

    # 처음 몇 개의 파일 형태 확인
    print("\nShape of the first 5 feature files:")
    for i in range(min(5, len(pairs))):
        feature_path = pairs[i][0]
        try:
            features = np.load(feature_path)
            print(f"  {os.path.basename(feature_path)} shape: {features.shape}")
        except FileNotFoundError:
            print(f"  Error loading {os.path.basename(feature_path)}: File not found.")
        except Exception as e:
            print(f"  Error loading {os.path.basename(feature_path)}: {e}")

    # 랜덤으로 몇 개의 파일 형태 확인
    print("\nShape of 5 random feature files:")
    random_indices = random.sample(range(len(pairs)), min(5, len(pairs)))
    for i in random_indices:
        feature_path = pairs[i][0]
        try:
            features = np.load(feature_path)
            print(f"  {os.path.basename(feature_path)} shape: {features.shape}")
        except FileNotFoundError:
             print(f"  Error loading {os.path.basename(feature_path)}: File not found.")
        except Exception as e:
            print(f"  Error loading {os.path.basename(feature_path)}: {e}")

    # 라벨 파일 형태도 선택적으로 확인 가능
    # print("\nShape of the first 5 label files:")
    # for i in range(min(5, len(pairs))):
    #     label_path = pairs[i][1]
    #     try:
    #         labels = np.load(label_path)
    #         print(f"  {os.path.basename(label_path)} shape: {labels.shape}")
    #     except FileNotFoundError:
    #          print(f"  Error loading {os.path.basename(label_path)}: File not found.")
    #     except Exception as e:
    #         print(f"  Error loading {os.path.basename(label_path)}: {e}")

This cell verifies the functionality of the `tf.data.Dataset` pipeline configured in the previous step. It shows the output shape of the dataset elements after mapping the loading function and after batching with padding, confirming that the data is being processed as expected.

In [ ]:
## -- tf.data.Dataset 동작 확인


print("--- Initial Dataset (paths) ---")
for feat_path, lab_path in ds_initial.take(5): # 처음 5개 요소만 확인
    print(f"Feature Path: {feat_path.numpy().decode('utf-8')}, Label Path: {lab_path.numpy().decode('utf-8')}")


ds_mapped = ds_initial.map(_load_npy, num_parallel_calls=tf.data.AUTOTUNE)

print("\n--- Dataset after map(_load_npy) ---")
for features, labels in ds_mapped.take(3): # 처음 3개 요소만 확인
    print(f"Features shape: {features.shape}, Labels shape: {labels.shape}")
    # 필요하다면 실제 데이터 값의 일부를 출력해볼 수도 있습니다.
    # print("Features example:", features.numpy().flatten()[:10])
    # print("Labels example:", labels.numpy())


ds_batched = ds_mapped.padded_batch(
    BATCH_SIZE,
    padded_shapes=([128, None, 1], [11]),
    padding_values=(tf.constant(0.0, dtype=tf.float32), tf.constant(0.0, dtype=tf.float32)),
    drop_remainder=False
)

print("\n--- Dataset after padded_batch ---")
for batch_features, batch_labels in ds_batched.take(2): # 처음 2개 배치만 확인
    print(f"Batch Features shape: {batch_features.shape}, Batch Labels shape: {batch_labels.shape}")
    # 배치 내의 데이터 값과 패딩 상태를 확인해볼 수 있습니다.
    # print("Batch Features example (first element, first few values):", batch_features.numpy()[0, :10, 0, 0])

This cell is a redundant check to ensure that the `pairs` list, containing the paths to the feature and label files, is correctly loaded and that the shapes and data types of randomly selected `.npy` files are as expected.

In [ ]:
import numpy as np
import random
import os

# pairs 리스트에서 임의의 features 및 label 파일 경로 선택
# pairs는 이전에 collect_pairs 함수를 통해 생성되었습니다.
if len(pairs) > 0:
    random_pair = random.choice(pairs)
    feature_path = random_pair[0]
    label_path = random_pair[1]

    print(f"Selected Feature File: {feature_path}")
    print(f"Selected Label File: {label_path}")

    try:
        # .npy 파일 로드
        features = np.load(feature_path)
        labels = np.load(label_path)

        # shape 출력
        print(f"Features shape: {features.shape}")
        print(f"Labels shape: {labels.shape}")

        # 데이터 타입 출력 (참고용)
        print(f"Features dtype: {features.dtype}")
        print(f"Labels dtype: {labels.dtype}")

    except FileNotFoundError:
        print("Error: One or both selected files not found. Please ensure the paths are correct and the files exist.")
    except Exception as e:
        print(f"An error occurred: {e}")

else:
    print("No pairs found in the list. Please ensure the data processing step completed successfully.")

This section defines the architecture of a Convolutional Neural Network (CNN) model for audio classification using TensorFlow/Keras. The model consists of convolutional layers for spatial feature extraction, followed by pooling, batch normalization, activation functions, dropout for regularization, and a final dense layer with softmax activation for classification.

In [ ]:
# CNN Model 구조 정의해보기
from tensorflow.keras import layers, models, callbacks, optimizers

#IRMAS 기본 Class 수는 11개
num_classes = 11

# 다음과 같은 구조로 저장되어 있으나, Feature은 DataLoader에 의해 3차원 변환됨(Channel = 1)
# Features shape: (128, 130)
# Labels shape: (11,)
# Features dtype: float32
# Labels dtype: float32

# 함수형으로 구현한 Model 사용해보기
def build_cnn(input_shape = (128, None, 1), num_classes= num_classes):
    inputs = layers.Input(shape=input_shape)

    # Block 1
    x = layers.Conv2D(32, (3,3), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D((2,2))(x)

    # Block 2
    x = layers.Conv2D(64, (3,3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D((2,2))(x)

    # Block 3
    x = layers.Conv2D(128, (3,3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D((2,2))(x)

    # Head
    x = layers.Dropout(0.3)(x)
    x = layers.GlobalAveragePooling2D()(x)   # 파라미터 적고 과적합 덜함
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

model = build_cnn()
model.summary()

This cell is a placeholder for the code to start the training process. The actual training loop is implemented in the subsequent K-Fold cross-validation cell.

In [ ]:
# 학습 시작 코드

This section implements K-Fold cross-validation to train and evaluate the defined CNN model. It splits the data into `n_splits` folds, trains a new model on the training folds, evaluates it on the validation fold, and records the loss and accuracy for each fold. It also includes Early Stopping to prevent overfitting and plots the training history for the first fold.

In [ ]:
# K-Fold 교차 검증 설정 및 학습/평가 코드

import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold
from tensorflow.keras import layers, models, callbacks, optimizers
import os

# K-Fold 설정
n_splits = 5  # 5-Fold 교차 검증
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42) # 데이터 순서를 섞고, 재현성을 위해 random_state 설정

# 데이터셋 로딩 및 전처리 함수 (이전에 정의된 _load_npy 함수와 padded_batch 사용)
# 여기서는 기존의 ds 구성 방식을 K-Fold 분할에 맞춰 수정합니다.

# K-Fold 결과를 저장할 리스트
fold_results = []
fold_histories = []

# 전체 데이터셋 (pairs)에서 feature_paths와 label_paths 분리
all_feature_paths = np.array([p[0] for p in pairs])
all_label_paths = np.array([p[1] for p in pairs])

# K-Fold 분할 시작
for fold, (train_index, val_index) in enumerate(kf.split(all_feature_paths)):
    print(f"\n--- Starting Fold {fold+1}/{n_splits} ---")

    # 현재 폴드의 훈련 및 검증 데이터 경로
    train_feature_paths = all_feature_paths[train_index]
    train_label_paths = all_label_paths[train_index]

    val_feature_paths = all_feature_paths[val_index]
    val_label_paths = all_label_paths[val_index]

    # 훈련 및 검증 데이터셋 생성
    train_ds = tf.data.Dataset.from_tensor_slices((train_feature_paths, train_label_paths))
    val_ds = tf.data.Dataset.from_tensor_slices((val_feature_paths, val_label_paths))

    # 기존 _load_npy 함수를 사용하여 데이터 로드 및 전처리 적용
    train_ds = (train_ds
                .shuffle(len(train_feature_paths)) # 훈련 데이터만 섞음
                .map(_load_npy, num_parallel_calls=tf.data.AUTOTUNE) # 병렬 로딩
                .padded_batch(BATCH_SIZE,
                              padded_shapes=([128, None, 1], [11]),
                              padding_values=(tf.constant(0.0, dtype=tf.float32), tf.constant(0.0, dtype=tf.float32)),
                              drop_remainder=False) # Batch 묵기
                .with_options(options)
                .prefetch(tf.data.AUTOTUNE))

    val_ds = (val_ds
              .map(_load_npy, num_parallel_calls=tf.data.AUTOTUNE) # 병렬 로딩
              .padded_batch(BATCH_SIZE,
                            padded_shapes=([128, None, 1], [11]),
                            padding_values=(tf.constant(0.0, dtype=tf.float32), tf.constant(0.0, dtype=tf.float32)),
                            drop_remainder=False) # Batch 묵기
              .with_options(options)
              .prefetch(tf.data.AUTOTUNE))


    # 새로운 모델 인스턴스 생성 (각 폴드마다 초기화된 모델 사용)
    # build_cnn 함수는 이전에 정의된 셀 (vK1fb-FsZwI3)에 있습니다.
    # input_shape은 데이터의 형태에 맞춰 ([128, None, 1])로 지정해야 합니다.
    model = build_cnn(input_shape=(128, None, 1))


    # 모델 컴파일
    model.compile(optimizer=optimizers.Adam(learning_rate=0.001), # 예시 학습률
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # 콜백 설정 (Early Stopping)
    # patience: 개선이 없을 때 몇 epoch을 더 기다릴지
    # restore_best_weights: 학습 중 가장 성능이 좋았던 가중치를 복원할지 여부
    early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


    # 모델 학습
    # steps_per_epoch와 validation_steps를 설정하면 더 정확한 진행률 확인 가능
    # train_steps = tf.data.experimental.cardinality(train_ds).numpy() # 데이터셋 크기를 알아내는 방법 중 하나
    # val_steps = tf.data.experimental.cardinality(val_ds).numpy()
    history = model.fit(train_ds,
                        # steps_per_epoch=train_steps,
                        epochs=50, # 충분히 큰 에포크 수를 설정하고 Early Stopping 활용
                        validation_data=val_ds,
                        # validation_steps=val_steps,
                        callbacks=[early_stopping],
                        verbose=1) # 학습 진행 상황 출력

    # 폴드 결과 평가
    print(f"\n--- Evaluating Fold {fold+1}/{n_splits} ---")
    loss, accuracy = model.evaluate(val_ds, verbose=0)
    print(f"Fold {fold+1} - Validation Loss: {loss:.4f}, Validation Accuracy: {accuracy:.4f}")

    fold_results.append({'loss': loss, 'accuracy': accuracy})
    fold_histories.append(history.history)

# K-Fold 교차 검증 결과 요약
print("\n--- K-Fold Cross-Validation Results ---")
avg_loss = np.mean([res['loss'] for res in fold_results])
avg_accuracy = np.mean([res['accuracy'] for res in fold_results])

print(f"Average Validation Loss: {avg_loss:.4f}")
print(f"Average Validation Accuracy: {avg_accuracy:.4f}")

# 각 폴드별 상세 결과 출력 (선택 사항)
# for i, res in enumerate(fold_results):
#     print(f"  Fold {i+1}: Loss={res['loss']:.4f}, Accuracy={res['accuracy']:.4f}")

# 가장 좋은 성능을 보인 폴드의 모델을 사용하거나, 전체 데이터로 재학습 등 후속 작업 진행 가능

# 평가 결과 시각화 (선택 사항)

import matplotlib.pyplot as plt

# 각 폴드의 학습 history에서 손실 및 정확도 가져오기
# 주의: Early Stopping으로 인해 각 폴드의 epoch 수가 다를 수 있습니다.
# 여기서는 각 폴드의 최종 성능만 확인하거나, history를 개별적으로 시각화할 수 있습니다.

# 예시: 각 폴드의 학습 및 검증 손실 그래프 그리기
# 모든 폴드의 history를 한 그래프에 그리면 복잡할 수 있습니다.
# 여기서는 예시로 첫 번째 폴드의 history를 그립니다.

if fold_histories:
    first_fold_history = fold_histories[0]
    plt.figure(figsize=(12, 6))

    # 손실 그래프
    plt.subplot(1, 2, 1)
    plt.plot(first_fold_history['loss'], label='Train Loss')
    plt.plot(first_fold_history['val_loss'], label='Val Loss')
    plt.title('Loss vs. Epochs (First Fold)')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # 정확도 그래프
    plt.subplot(1, 2, 2)
    plt.plot(first_fold_history['accuracy'], label='Train Accuracy')
    plt.plot(first_fold_history['val_accuracy'], label='Val Accuracy')
    plt.title('Accuracy vs. Epochs (First Fold)')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

This section defines the architecture of a Convolutional Recurrent Neural Network (CRNN) model, which combines CNN layers for spatial feature extraction and Bidirectional LSTM layers for temporal sequence modeling. This type of model is often effective for audio classification tasks involving time-varying features. It includes layers for convolution, pooling, batch normalization, dropout, permutation and TimeDistributed Flatten to prepare data for the RNN, and a final dense layer for classification.

In [ ]:
#Sequential 관련 모델들 시험해보기

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, optimizers

# IRMAS 기본 Class 수는 11개
num_classes = 11

# 기존 DataLoader에서 생성되는 데이터 형태: (128, None, 1) -> 배치 후 (BATCH_SIZE, 128, Max_Time, 1)
input_shape = (128, None, 1) # Mel Spectrogram (n_mels, time_steps, channels)

def build_crnn(input_shape=input_shape, num_classes=num_classes):
    inputs = layers.Input(shape=input_shape)

    # --- CNN 부분 ---
    # Mel Spectrogram에서 공간적 특징 추출
    x = layers.Conv2D(32, (3, 3), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D((2, 2))(x) # 주파수 축과 시간 축 모두 압축

    x = layers.Conv2D(64, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D((2, 2))(x) # 주파수 축과 시간 축 모두 압축

    x = layers.Conv2D(128, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    # 마지막 MaxPooling은 시간 축만 압축하거나, Global Pooling을 고려할 수 있습니다.
    # 여기서는 시간 축 압축을 위해 (1, 2) 또는 (1, 4) 등을 사용해 봅시다.
    # 주파수 축은 RNN 입력으로 넘어가기 전에 납작하게 만들 것이므로, 여기서 너무 압축하지 않는 것이 좋습니다.
    # 따라서 마지막 MaxPooling은 시간 축에 더 집중하거나 (1, P) 또는 사용하지 않을 수 있습니다.
    # 일반적인 오디오 CRNN에서는 CNN 후 특징 맵을 시간 축 방향으로 펼칩니다.
    # 시간 축 압축을 위해 (1, 2)를 사용해 보겠습니다.
    x = layers.MaxPooling2D((1, 2))(x) # 시간 축만 2배 압축

    # --- 특징 맵을 RNN 입력 형태로 변환 ---
    # CNN 출력 형태: (Batch, F', T', C')
    # RNN 입력 형태: (Batch, T', F'*C')
    # tf.keras.layers.Permute 또는 Reshape, TimeDistributed 등을 사용할 수 있습니다.
    # 가변 길이 시간 축 처리를 위해 Reshape보다는 TimeDistributed를 사용하는 것이 더 유연할 수 있으나,
    # 여기서는 GlobalAveragePooling2D와 유사하게 공간 축(주파수)을 납작하게 만들고 시간 축을 시퀀스로 사용합니다.

    # CNN 출력 shape 확인: (Batch, F', T', C') -> 예를 들어 (None, 16, None, 128) after max pooling (1,2)
    # 주파수 축 (F')을 특징 벡터로 사용하고, 시간 축 (T')을 시퀀스 길이로 사용합니다.
    # Reshape을 사용하여 (Batch, T', F' * C') 형태로 만듭니다.
    # 가변 시간 축 때문에 Reshape이 바로 작동하지 않을 수 있습니다.
    # 대신, TimeDistributed(Flatten) 또는 Lambda 레이어를 사용하여 각 타임스텝의 특징을 납작하게 만듭니다.
    # 여기서는 Lambda를 사용하여 (Batch, T', F' * C') 형태로 변환합니다.
    # Lambda 레이어는 사용자 정의 함수를 레이어로 추가할 수 있게 해줍니다.
    # 람다 함수는 입력 텐서 x를 받아서 원하는 변환을 수행합니다.
    # x.shape: (None, F', T', C')
    # 목표 shape: (None, T', F' * C')
    # 축 순서를 변경하고 Flatten을 적용합니다.
    # Permute를 먼저 사용해 (Batch, T', F', C') 로 만든 후 Reshape 또는 Flatten
    # TimeDistributed(Flatten())을 사용하면 각 타임스텝에 대해 Flatten을 적용합니다.
    # (Batch, T', F', C') -> TimeDistributed(Flatten) -> (Batch, T', F'*C')

    # CNN 출력 shape: (Batch, F', T', C')
    # F' = 128 / (2*2) = 32 (MaxPooling2D (2,2) 두 번)
    # T' = None / (2*2*2) = None / 8 (MaxPooling2D (2,2) 두 번, (1,2) 한 번)
    # C' = 128
    # CNN 출력 shape 예시: (None, 32, None, 128)

    # TimeDistributed Flatten을 사용하기 위해 축 순서를 변경
    # (Batch, F', T', C') -> (Batch, T', F', C')
    x = layers.Permute((2, 1, 3))(x) # (Batch, T', F', C')

    # 각 타임스텝에 대해 Flatten 적용
    # (Batch, T', F', C') -> (Batch, T', F'*C')
    # TimeDistributed 레이어는 입력의 첫 번째 차원(배치 차원 다음)을 시퀀스 차원으로 간주하고
    # 그 이후 레이어를 각 시퀀스 스텝에 독립적으로 적용합니다.
    x = layers.TimeDistributed(layers.Flatten())(x) # (Batch, T', features_per_timestep)
    # features_per_timestep = F' * C' = 32 * 128 = 4096

    # --- RNN 부분 ---
    # 시간적 순서 모델링
    # Bidirectional LSTM을 사용하여 양방향 문맥 학습
    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x) # return_sequences=True는 다음 LSTM/GRU 레이어로 연결할 때 필요
    x = layers.Dropout(0.3)(x)
    x = layers.Bidirectional(layers.LSTM(128))(x) # 마지막 LSTM은 시퀀스 전체를 요약 (return_sequences=False)
    x = layers.Dropout(0.3)(x)

    # --- Head (분류) ---
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model

# CRNN 모델 생성 및 요약 출력
crnn_model = build_crnn()
crnn_model.summary()

This cell implements K-Fold cross-validation to train and evaluate the defined CRNN model. Similar to the CNN evaluation cell, it splits the data, trains the CRNN model on each fold, evaluates its performance, and summarizes the results. It also includes Early Stopping and plots the training history for the first fold.

In [ ]:
# CRNN 모델 K-Fold 교차 검증 설정 및 학습/평가 코드

import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold
from tensorflow.keras import layers, models, callbacks, optimizers
import os

# K-Fold 설정
n_splits = 5  # 5-Fold 교차 검증
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42) # 데이터 순서를 섞고, 재현성을 위해 random_state 설정

# 데이터셋 로딩 및 전처리 함수 (_load_npy 함수와 padded_batch는 이전 셀에 정의되어 있다고 가정)
# 여기서는 기존의 ds 구성 방식을 K-Fold 분할에 맞춰 수정합니다.

# K-Fold 결과를 저장할 리스트
crnn_fold_results = []
crnn_fold_histories = []

# 전체 데이터셋 (pairs)에서 feature_paths와 label_paths 분리
# pairs 리스트는 이전 셀 (3d184f11)에서 로컬 경로를 사용하여 이미 생성되었다고 가정합니다.
if 'pairs' not in locals():
    print("Error: 'pairs' list not found. Please run the cell that collects file pairs using LOCAL_DIR first.")
else:
    all_feature_paths = np.array([p[0] for p in pairs])
    all_label_paths = np.array([p[1] for p in pairs])

    # K-Fold 분할 시작
    for fold, (train_index, val_index) in enumerate(kf.split(all_feature_paths)):
        print(f"\n--- Starting CRNN Fold {fold+1}/{n_splits} ---")

        # 현재 폴드의 훈련 및 검증 데이터 경로
        train_feature_paths = all_feature_paths[train_index]
        train_label_paths = all_label_paths[train_index]

        val_feature_paths = all_feature_paths[val_index]
        val_label_paths = all_label_paths[val_index]

        # 훈련 및 검증 데이터셋 생성 (로컬 경로 사용)
        train_ds = tf.data.Dataset.from_tensor_slices((train_feature_paths, train_label_paths))
        val_ds = tf.data.Dataset.from_tensor_slices((val_feature_paths, val_label_paths))

        # 기존 _load_npy 함수를 사용하여 데이터 로드 및 전처리 적용
        # _load_npy 함수는 이전 셀 (3d184f11)에 정의되어 있습니다.
        train_ds = (train_ds
                    .shuffle(len(train_feature_paths)) # 훈련 데이터만 섞음
                    .map(_load_npy, num_parallel_calls=tf.data.AUTOTUNE) # 병렬 로딩
                    .padded_batch(BATCH_SIZE, # BATCH_SIZE는 이전 셀 (3d184f11)에 정의
                                  padded_shapes=([128, None, 1], [11]), # padded_shapes는 이전 셀 (3d184f11)에 정의
                                  padding_values=(tf.constant(0.0, dtype=tf.float32), tf.constant(0.0, dtype=tf.float32)),
                                  drop_remainder=False) # Batch 묵기
                    .with_options(options) # options는 이전 셀 (3d184f11)에 정의
                    .prefetch(tf.data.AUTOTUNE))

        val_ds = (val_ds
                  .map(_load_npy, num_parallel_calls=tf.data.AUTOTUNE) # 병렬 로딩
                  .padded_batch(BATCH_SIZE,
                                padded_shapes=([128, None, 1], [11]),
                                padding_values=(tf.constant(0.0, dtype=tf.float32), tf.constant(0.0, dtype=tf.float32)),
                                drop_remainder=False) # Batch 묵기
                  .with_options(options)
                  .prefetch(tf.data.AUTOTUNE))


        # 새로운 CRNN 모델 인스턴스 생성 (각 폴드마다 초기화된 모델 사용)
        # build_crnn 함수는 이전 셀 (4d4e6a28)에 정의되어 있습니다.
        # input_shape은 데이터의 형태에 맞춰 ([128, None, 1])로 지정해야 합니다.
        crnn_model = build_crnn(input_shape=(128, None, 1))


        # 모델 컴파일
        crnn_model.compile(optimizer=optimizers.Adam(learning_rate=0.001), # 예시 학습률
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        # 콜백 설정 (Early Stopping)
        # patience: 개선이 없을 때 몇 epoch을 더 기다릴지
        # restore_best_weights: 학습 중 가장 성능이 좋았던 가중치를 복원할지 여부
        early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


        # 모델 학습
        history = crnn_model.fit(train_ds,
                            epochs=50, # 충분히 큰 에포크 수를 설정하고 Early Stopping 활용
                            validation_data=val_ds,
                            callbacks=[early_stopping],
                            verbose=1) # 학습 진행 상황 출력

        # 폴드 결과 평가
        print(f"\n--- Evaluating CRNN Fold {fold+1}/{n_splits} ---")
        loss, accuracy = crnn_model.evaluate(val_ds, verbose=0)
        print(f"CRNN Fold {fold+1} - Validation Loss: {loss:.4f}, Validation Accuracy: {accuracy:.4f}")

        crnn_fold_results.append({'loss': loss, 'accuracy': accuracy})
        crnn_fold_histories.append(history.history)

    # K-Fold 교차 검증 결과 요약
    print("\n--- CRNN K-Fold Cross-Validation Results ---")
    avg_loss = np.mean([res['loss'] for res in crnn_fold_results])
    avg_accuracy = np.mean([res['accuracy'] for res in crnn_fold_results])

    print(f"CRNN Average Validation Loss: {avg_loss:.4f}")
    print(f"CRNN Average Validation Accuracy: {avg_accuracy:.4f}")

    # 각 폴드별 상세 결과 출력 (선택 사항)
    # for i, res in enumerate(crnn_fold_results):
    #     print(f"  CRNN Fold {i+1}: Loss={res['loss']:.4f}, Accuracy={res['accuracy']:.4f}")

    # 평가 결과 시각화 (선택 사항)

    import matplotlib.pyplot as plt

    if crnn_fold_histories:
        # 모든 폴드의 history를 한 그래프에 그리면 복잡할 수 있습니다.
        # 여기서는 예시로 첫 번째 폴드의 history를 그립니다.
        first_fold_history = crnn_fold_histories[0]
        plt.figure(figsize=(12, 6))

        # 손실 그래프
        plt.subplot(1, 2, 1)
        plt.plot(first_fold_history['loss'], label='Train Loss')
        plt.plot(first_fold_history['val_loss'], label='Val Loss')
        plt.title('CRNN Loss vs. Epochs (First Fold)')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)

        # 정확도 그래프
        plt.subplot(1, 2, 2)
        plt.plot(first_fold_history['accuracy'], label='Train Accuracy')
        plt.plot(first_fold_history['val_accuracy'], label='Val Accuracy')
        plt.title('CRNN Accuracy vs. Epochs (First Fold)')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.grid(True)

        plt.tight_layout()
        plt.show()

This is a placeholder cell indicating that the next steps might involve experimenting with Transformer models for audio classification.

In [ ]:
# 이 다음은 Transformer 모델 시험...